In [1]:
from libs.connector.postgres_connector import PostgresToJsonConnector
from libs.handler.postgres_handler import PostgresHandler
from libs.handler.json_handler import JsonHandler
from libs.log_handler.log_handler import LogHandler

from libs.workers.algo.exact_name_url import deduplicate_exact_match_name_url
from libs.dataframes.to_types import to_list_of_dicts

import pandas as pd


log_handler = PostgresHandler(
    env_prefix = 'POSTGRES'
)

logger = LogHandler(db_handler=log_handler)


input_qry = '''
    SELECT
        id,
        name,
        website
    FROM organization
    '''

input_handler = PostgresHandler(
    query = input_qry,
    schema = 'connect211_wa211_hsds3',
    env_prefix = 'POSTGRES'
)

output_handler = JsonHandler()


In [2]:

conn = PostgresToJsonConnector(
    input_handler=input_handler,
    output_handler=output_handler,
    log_handler=logger,
)

In [3]:


DEDUPLICATION_STEPS = {
    'dedupe_io_names_addresses': 'libs.workers.ml.names_addresses.names_addresses',
    'deduplicate_exact_match_name_url': 'libs.workers.algo.exact_name_url',
}
steps = dict()
my_steps = ['deduplicate_exact_match_name_url']
for step in my_steps:
    if DEDUPLICATION_STEPS.get(step):
        steps[step] = DEDUPLICATION_STEPS[step]


job_id = conn.mutate(conn.parse_steps(steps))

In [4]:
print (job_id)

05420fd9-fbdd-461b-9133-95e6b8b1ec6b
